In [20]:
# import library yang dibutuhkan
import tensorflow as tf
from sklearn.model_selection import train_test_split
import shutil
import zipfile,os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image as keras_image
import matplotlib.pyplot as plt
import cv2
from rembg.bg import remove
import easygui
from PIL import Image
from concurrent.futures import ProcessPoolExecutor
import wget

print(tf.__version__)

2.18.0


In [21]:
# check directory
if os.path.exists('./train'):
    print('Direktori sudah ada isinya')

Direktori sudah ada isinya


# Data Preprocessing

In [22]:
# import os
# from rembg import remove
# from PIL import Image

# def remove_background(input_path, output_path):
#     try:
#         input_image = Image.open(input_path) # ngebuka filenya
#         output_image = remove(input_image) # ngehapus background -> library rembg
#         output_image.save(output_path) # nyimpen yang udah diapus ke background
#     except Exception as e: # kalo gagal nyimpen yg removed background disimpen di log
#         with open("error_log.txt", "a") as log_file:
#             log_file.write(f"Error processing {input_path}: {e}\n")

# def remove_background_serial(input_dir, output_dir):  # loop remove semua
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for filename in os.listdir(input_dir):
#         input_path = os.path.join(input_dir, filename)
#         output_path = os.path.join(output_dir, filename) # masukkin gambarnya
#         remove_background(input_path, output_path)

# # Use serial processing for background removal
# original_dataset_dir = './train'  # Replace with your dataset directory
# processed_dataset_dir = './processed'
# categories = ['fall', 'non_fall']

# subjects = [f for f in os.listdir('./train') if os.path.isdir(os.path.join('./train', f))]

# for subject in subjects:
#     for category in categories:
#         input_dir = os.path.join('./train', subject, category) # file train
#         output_dir = os.path.join('./processed', subject, category) # file buat nyimpen hasil

#         if os.path.exists(input_dir):
#             remove_background_serial(input_dir, output_dir)

In [23]:
import os
from rembg import remove
from PIL import Image

def remove_background(input_path, output_path):
    try:
        input_image = Image.open(input_path) # ngebuka filenya
        output_image = remove(input_image) # ngehapus background -> library rembg
        output_image.save(output_path) # nyimpen yang udah diapus ke background
    except Exception as e: # kalo gagal nyimpen yg removed background disimpen di log
        with open("error_log.txt", "a") as log_file:
            log_file.write(f"Error processing {input_path}: {e}\n")

# serial processing for background removal
def remove_background_serial(input_dir, output_dir): # loop remove background semua
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename) # masukkin gambarnya
        remove_background(input_path, output_path)

# path dataset asli dan hasil
original_dataset_dir = './train'  # file train (dataset asli)
processed_dataset_dir = './processed'  # file buat nyimpen hasil
categories = ['fall', 'non_fall']

# penghapusan background untuk setiap kategori dalam setiap subdirektori (subject)
subjects = [f for f in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, f))]

for subject in subjects:
    for category in categories:
        input_dir = os.path.join(original_dataset_dir, subject, category)
        output_dir = os.path.join(processed_dataset_dir, subject, category)

        if os.path.exists(input_dir):
            remove_background_serial(input_dir, output_dir)

In [24]:
# # membuat direktori untuk data training dan data validasi
# base_dir = './processed'
# train_dir = os.path.join(base_dir, 'train')
# validation_dir = os.path.join(base_dir, 'val')

# # membuat direktori untuk data training 
# train_fall_dir = os.path.join(train_dir, 'fall') 
# train_non_fall_dir = os.path.join(train_dir, 'non_fall')

# # membuat direktori untuk data validasi
# validation_fall_dir = os.path.join(validation_dir, 'fall')
# validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

# # membuat direktori untuk data training dan data validasi
# if not os.path.exists(train_dir):
#     os.mkdir(train_dir)
# if not os.path.exists(validation_dir):
#     os.mkdir(validation_dir)

# if not os.path.exists(train_fall_dir):
#     os.mkdir(train_fall_dir)
# if not os.path.exists(train_non_fall_dir):
#     os.mkdir(train_non_fall_dir)

# if not os.path.exists(validation_fall_dir):
#     os.mkdir(validation_fall_dir)
# if not os.path.exists(validation_non_fall_dir):
#     os.mkdir(validation_non_fall_dir)

In [35]:
# membuat direktori data training dan validasi
base_dir = './processed_split'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# membuat subdirektori untuk kategori (data training)
train_fall_dir = os.path.join(train_dir, 'fall')
train_non_fall_dir = os.path.join(train_dir, 'non_fall')

# membuat subdirektori untuk kategori (data validasi)
validation_fall_dir = os.path.join(validation_dir, 'fall')
validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

# membuat semua direktori jika belum ada
for directory in [train_dir, validation_dir, train_fall_dir, train_non_fall_dir, validation_fall_dir, validation_non_fall_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# membagi dataset ke dalam train dan validation
train_fall = []
validation_fall = []
train_non_fall = []
validation_non_fall = []

for subject in subjects:
    # fall
    fall_dir = os.path.join(processed_dataset_dir, subject, 'fall')
    fall_images = os.listdir(fall_dir) if os.path.exists(fall_dir) else []
    
    if len(fall_images) > 0:  # cek apakah direktori tidak kosong
        train, val = train_test_split(fall_images, test_size=0.4, random_state=None)
        train_fall += [(subject, 'fall', img) for img in train]
        validation_fall += [(subject, 'fall', img) for img in val]
    else:
        print("check kosong")

    # non-fall
    non_fall_dir = os.path.join(processed_dataset_dir, subject, 'non_fall')
    non_fall_images = os.listdir(non_fall_dir) if os.path.exists(non_fall_dir) else []
    
    if len(non_fall_images) > 0:  # cek apakah direktori tidak kosong
        train, val = train_test_split(non_fall_images, test_size=0.4, random_state=None)
        train_non_fall += [(subject, 'non_fall', img) for img in train]
        validation_non_fall += [(subject, 'non_fall', img) for img in val]
    else:
        print("check kosong")


check kosong
check kosong
check kosong
check kosong
check kosong
check kosong
check kosong
check kosong


In [27]:
# membagi data train dan data validation dengan perbandingan 60:40 

subjects = [f for f in os.listdir('./processed') if os.path.isdir(os.path.join('./processed', f))]
train_fall = []
validation_fall = []

for subject in subjects:
    fall_images = os.listdir(os.path.join('./processed', subject, 'fall'))
    train, val = train_test_split(fall_images, test_size=0.4, random_state=None)
    train_fall += [os.path.join(subject, 'fall', img) for img in train]
    validation_fall += [os.path.join(subject, 'fall', img) for img in val]

# train test split jika direktori belum ada isinya
if len(os.listdir('./train/fall')) == 0 and len(os.listdir('./train/non_fall')) == 0 :
    # fall
    train_fall = os.listdir('./processed/fall')
    validation_fall = os.listdir('./processed/fall')
    train_fall, validation_fall = train_test_split(train_fall, test_size = 0.4, random_state=None)

    # non_fall
    train_non_fall = os.listdir('./processed/non_fall')
    validation_non_fall = os.listdir('./processed/non_fall')
    train_non_fall, validation_non_fall = train_test_split(train_non_fall, test_size = 0.4, random_state=None)

    # Memindahkan file gambar ke direktori data training (train)
    for image in train_fall:
        shutil.move(os.path.join('./processed/fall', image), os.path.join(train_fall_dir, image))
    for image in train_non_fall:
        shutil.move(os.path.join('./processed/non_fall', image), os.path.join(train_non_fall_dir, image))

    # Memindahkan file gambar ke direktori data validasi (validation)
    for image in validation_fall:
        shutil.move(os.path.join('./processed/fall', image), os.path.join(validation_fall_dir, image))
    for image in validation_non_fall:
        shutil.move(os.path.join('./processed/non_fall', image), os.path.join(validation_non_fall_dir, image))
else:
    print('Direktori sudah ada isinya')

In [28]:
# memindahkan file ke direktori training dan validasi
for subject, category, image in train_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(train_fall_dir, image)
    shutil.move(src, dst)

for subject, category, image in validation_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(validation_fall_dir, image)
    shutil.move(src, dst)

for subject, category, image in train_non_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(train_non_fall_dir, image)
    shutil.move(src, dst)

for subject, category, image in validation_non_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(validation_non_fall_dir, image)
    shutil.move(src, dst)

print("Dataset berhasil diproses dan dipisahkan ke dalam direktori training dan validasi.")

Dataset berhasil diproses dan dipisahkan ke dalam direktori training dan validasi.


In [29]:
# Membagi data train dan data validation dengan perbandingan 60:40

# Train test split jika direktori belum ada isinya
if all(len(os.listdir(os.path.join('./processed_split/train', category))) == 0 for category in ['fall', 'non_fall']):
    train_fall = []
    validation_fall = []
    train_non_fall = []
    validation_non_fall = []

    # Iterasi untuk setiap subject
    for subject in subjects:
        # Fall
        fall_dir = os.path.join(processed_dataset_dir, subject, 'fall')
        fall_images = os.listdir(fall_dir) if os.path.exists(fall_dir) else []

        if len(fall_images) > 0:  # Cek apakah direktori tidak kosong
            train, val = train_test_split(fall_images, test_size=0.4, random_state=None)
            train_fall += [(subject, 'fall', img) for img in train]
            validation_fall += [(subject, 'fall', img) for img in val]

        # Non-fall
        non_fall_dir = os.path.join(processed_dataset_dir, subject, 'non_fall')
        non_fall_images = os.listdir(non_fall_dir) if os.path.exists(non_fall_dir) else []

        if len(non_fall_images) > 0:  # Cek apakah direktori tidak kosong
            train, val = train_test_split(non_fall_images, test_size=0.4, random_state=None)
            train_non_fall += [(subject, 'non_fall', img) for img in train]
            validation_non_fall += [(subject, 'non_fall', img) for img in val]

    # Memindahkan file gambar ke direktori data training (train)
    for subject, category, image in train_fall:
        src = os.path.join(processed_dataset_dir, subject, category, image)
        dst = os.path.join(train_fall_dir, image)
        shutil.move(src, dst)

    for subject, category, image in train_non_fall:
        src = os.path.join(processed_dataset_dir, subject, category, image)
        dst = os.path.join(train_non_fall_dir, image)
        shutil.move(src, dst)

    # Memindahkan file gambar ke direktori data validasi (validation)
    for subject, category, image in validation_fall:
        src = os.path.join(processed_dataset_dir, subject, category, image)
        dst = os.path.join(validation_fall_dir, image)
        shutil.move(src, dst)

    for subject, category, image in validation_non_fall:
        src = os.path.join(processed_dataset_dir, subject, category, image)
        dst = os.path.join(validation_non_fall_dir, image)
        shutil.move(src, dst)

else:
    print('Direktori sudah ada isinya')


In [31]:
# menampilkan nama kelas yang ada pada train
os.listdir('./processed/train')

['non_fall', 'fall']

In [32]:
# menampilkan nama kelas yang ada pada train
os.listdir('./processed/val')

['non_fall', 'fall']

# Data Modelling

In [ ]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from sklearn.model_selection import train_test_split
from rembg import remove
from PIL import Image

# 1. Fungsi Preprocessing Gambar
def preprocess_img(img):
    img = cv2.GaussianBlur(img, (5, 5), 0)  # Menghaluskan gambar
    img = tf.image.adjust_contrast(img, 2)  # Menyesuaikan kontras gambar
    return img

# 2. Fungsi Penghapusan Background
def remove_background(input_path, output_path):
    try:
        input_image = Image.open(input_path)
        output_image = remove(input_image)
        output_image.save(output_path)
    except Exception as e:
        with open("error_log.txt", "a") as log_file:
            log_file.write(f"Error processing {input_path}: {e}\n")

def remove_background_serial(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        remove_background(input_path, output_path)

# 3. Path Dataset
original_dataset_dir = './train'
processed_dataset_dir = './processed'
categories = ['fall', 'non_fall']

# Memproses dataset untuk setiap subjek dan kategori
subjects = [f for f in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, f))]
for subject in subjects:
    for category in categories:
        input_dir = os.path.join(original_dataset_dir, subject, category)
        output_dir = os.path.join(processed_dataset_dir, subject, category)
        if os.path.exists(input_dir):
            remove_background_serial(input_dir, output_dir)

# 4. Membagi Data Train dan Validation
base_dir = './processed_split'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
train_fall_dir = os.path.join(train_dir, 'fall')
train_non_fall_dir = os.path.join(train_dir, 'non_fall')
validation_fall_dir = os.path.join(validation_dir, 'fall')
validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

for directory in [train_dir, validation_dir, train_fall_dir, train_non_fall_dir, validation_fall_dir, validation_non_fall_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

train_fall = []
validation_fall = []
train_non_fall = []
validation_non_fall = []

for subject in subjects:
    fall_dir = os.path.join(processed_dataset_dir, subject, 'fall')
    fall_images = os.listdir(fall_dir) if os.path.exists(fall_dir) else []
    if len(fall_images) > 0:
        train, val = train_test_split(fall_images, test_size=0.4, random_state=None)
        train_fall += [(subject, 'fall', img) for img in train]
        validation_fall += [(subject, 'fall', img) for img in val]

    non_fall_dir = os.path.join(processed_dataset_dir, subject, 'non_fall')
    non_fall_images = os.listdir(non_fall_dir) if os.path.exists(non_fall_dir) else []
    if len(non_fall_images) > 0:
        train, val = train_test_split(non_fall_images, test_size=0.4, random_state=None)
        train_non_fall += [(subject, 'non_fall', img) for img in train]
        validation_non_fall += [(subject, 'non_fall', img) for img in val]

for subject, category, image in train_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(train_fall_dir, image)
    shutil.move(src, dst)

for subject, category, image in validation_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(validation_fall_dir, image)
    shutil.move(src, dst)

for subject, category, image in train_non_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(train_non_fall_dir, image)
    shutil.move(src, dst)

for subject, category, image in validation_non_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(validation_non_fall_dir, image)
    shutil.move(src, dst)

# 5. ImageDataGenerator untuk Training dan Validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    preprocessing_function=preprocess_img,
    brightness_range=[0.5, 1.5],
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# 6. Load dan Augmentasi Gambar untuk Visualisasi
img_path = './processed_split/train/fall/example_image.jpg'  # Ganti dengan path gambar yang tersedia
img = tf.keras.preprocessing.image.load_img(img_path)
x = tf.keras.preprocessing.image.img_to_array(img)
x = x.reshape((1,) + x.shape)

plt.figure(figsize=(10, 10))
plt.suptitle('Augmented Images')

i = 0
for batch in train_datagen.flow(x, batch_size=1):
    plt.subplot(3, 3, i + 1)
    plt.imshow(array_to_img(batch[0]))
    i += 1
    if i >= 6:
        break

plt.show()

print("Proses selesai. Dataset telah diproses dan dibagi menjadi train dan validation.")
